In [11]:
%pip install torch torchvision
%pip install trimesh numpy scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import necessary libraries

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import json
import trimesh
import os
import random
import shutil

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Read data

In [14]:
data = pd.read_csv('/home/aalab/Desktop/tripoFT/metadata.csv')

In [15]:
print("The first few rows of the DataFrame:")
print(data.head())

print("\nData info:")
print(data.info())

print("\nData columns:")
print(data.columns)

print("\nData described:")
print(data.describe())

The first few rows of the DataFrame:
                                 filename      size  \
0  cadbury_dairy_milk_chocolate_piece.glb    152796   
1          bonnie_melted_chocolate_ar.glb   8285860   
2              chocobar_ice_cream (1).glb   4499300   
3                           ice_cream.glb  12444648   
4                   chocolate_truffle.glb   2084100   

                                         object_path  
0  /home/aalab/Desktop/tripoFT/Dataset/cadbury_da...  
1  /home/aalab/Desktop/tripoFT/Dataset/bonnie_mel...  
2  /home/aalab/Desktop/tripoFT/Dataset/chocobar_i...  
3  /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb  
4  /home/aalab/Desktop/tripoFT/Dataset/chocolate_...  

Data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     145 non-null    object
 1   size         145 non-null    int64 
 2   object_path  145 

Clean Data

In [16]:
data.drop_duplicates(inplace=True)

# Drop rows with missing values in specific columns
columns_with_missing_values = ['filename', 'size', 'object_path']
data.dropna(subset=columns_with_missing_values, inplace=True)

# Convert columns to appropriate data types
data['filename'] = data['filename'].astype(str)
data['size'] = data['size'].astype(str)
data['object_path'] = data['object_path'].astype(str)

# Print cleaned DataFrame
print(data)

                                   filename      size  \
0    cadbury_dairy_milk_chocolate_piece.glb    152796   
1            bonnie_melted_chocolate_ar.glb   8285860   
2                chocobar_ice_cream (1).glb   4499300   
3                             ice_cream.glb  12444648   
4                     chocolate_truffle.glb   2084100   
..                                      ...       ...   
140                     strawberry_cake.glb  25159596   
141                jiggly_bunny_pudding.glb   1026416   
142            kitchen_sink_fiesta_5149.glb  26103980   
143   easter_egg_2024_marbled_chocolate.glb   3743544   
144              paleta_payaso_lollypop.glb    439920   

                                           object_path  
0    /home/aalab/Desktop/tripoFT/Dataset/cadbury_da...  
1    /home/aalab/Desktop/tripoFT/Dataset/bonnie_mel...  
2    /home/aalab/Desktop/tripoFT/Dataset/chocobar_i...  
3    /home/aalab/Desktop/tripoFT/Dataset/ice_cream.glb  
4    /home/aalab/Desktop/tripo

Convert 3D Objects into a Usable Format that our model can work with, such as voxel grids. 

In [17]:
import os
import numpy as np
import trimesh

def convert_glb_to_tensors(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop over all .glb files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.glb'):
            # Load the .glb file using trimesh
            file_path = os.path.join(input_folder, file_name)
            mesh = trimesh.load(file_path, force='mesh')

            # Extract vertices and faces
            vertices = mesh.vertices
            faces = mesh.faces

            # Combine vertices and faces into a single tensor
            combined_tensor = {
                'vertices': vertices,
                'faces': faces
            }

            # Save the tensor to a .npz file
            tensor_file_name = file_name.replace('.glb', '.npz')
            tensor_file_path = os.path.join(output_folder, tensor_file_name)
            np.savez(tensor_file_path, **combined_tensor)

            print(f'Successfully processed and saved: {tensor_file_name}')

input_folder = '/home/aalab/Desktop/tripoFT/Dataset'
output_folder = '/home/aalab/Desktop/tripoFT/tensors'
convert_glb_to_tensors(input_folder, output_folder)


Successfully processed and saved: cadbury_dairy_milk_chocolate_piece.npz
Successfully processed and saved: bonnie_melted_chocolate_ar.npz
Successfully processed and saved: chocobar_ice_cream (1).npz
Successfully processed and saved: ice_cream.npz
Successfully processed and saved: chocolate_truffle.npz
Successfully processed and saved: oreo.npz
Successfully processed and saved: heart_cake.npz
Successfully processed and saved: candy_vending_machine.npz
Successfully processed and saved: gummy_bear_bull.npz
Successfully processed and saved: valentines_heart.npz
Successfully processed and saved: beardpapas_smores_puff__scan.npz
Successfully processed and saved: doughnut_pack.npz
Successfully processed and saved: cc0_-_candies.npz
Successfully processed and saved: biscuit.npz
Successfully processed and saved: hot_fudge_warmer.npz
Successfully processed and saved: chocolate_milkshake.npz
Successfully processed and saved: vegan_icecream.npz
Successfully processed and saved: power_shake_chocola

In [20]:
import os
import numpy as np

def view_tensor_shapes(folder_path):
    # Loop over all .npz files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.npz'):
            file_path = os.path.join(folder_path, file_name)
            
            # Load the tensor from the .npz file
            tensor_data = np.load(file_path)
            vertices = tensor_data['vertices']
            faces = tensor_data['faces']
            
            # Print the shapes of the vertices and faces
            print(f"File: {file_name}")
            print(f"  Vertices shape: {vertices.shape}")
            print(f"  Faces shape: {faces.shape}\n")

# Path to the folder containing the .npz files
folder_path = '/home/aalab/Desktop/tripoFT/tensors'
view_tensor_shapes(folder_path)


File: cup_cake.npz
  Vertices shape: (4741, 3)
  Faces shape: (8862, 3)

File: brownie_square.npz
  Vertices shape: (25260, 3)
  Faces shape: (48314, 3)

File: ice_cream (1).npz
  Vertices shape: (499, 3)
  Faces shape: (528, 3)

File: schokokuchen_lina_2024.npz
  Vertices shape: (15420, 3)
  Faces shape: (24999, 3)

File: hot_chocolate_with_whipped_cream.npz
  Vertices shape: (820, 3)
  Faces shape: (416, 3)

File: doughnut_pack.npz
  Vertices shape: (2980976, 3)
  Faces shape: (993662, 3)

File: simple_chocolate_bar.npz
  Vertices shape: (949, 3)
  Faces shape: (1304, 3)

File: hershey_kiss.npz
  Vertices shape: (700, 3)
  Faces shape: (1208, 3)

File: chocobar_ice_cream.npz
  Vertices shape: (7509, 3)
  Faces shape: (8592, 3)

File: strawberry_cake_low_poly.npz
  Vertices shape: (296, 3)
  Faces shape: (398, 3)

File: paper_cup_perfect.npz
  Vertices shape: (3684, 3)
  Faces shape: (3764, 3)

File: chocolate_buttercream_cupcake.npz
  Vertices shape: (513870, 3)
  Faces shape: (17129

Data Augmentation

In [ ]:
# import os
# import numpy as np
# import trimesh
# import random

# def augment_vertices(vertices):
#     """Apply random transformations to the vertices."""
#     # Random rotation
#     angle = np.random.uniform(0, 2 * np.pi)
#     axis = np.random.uniform(-1, 1, 3)
#     axis /= np.linalg.norm(axis)
#     rotation_matrix = trimesh.transformations.rotation_matrix(angle, axis)[:3, :3]
    
#     # Random scaling
#     scale = np.random.uniform(0.9, 1.1)
    
#     # Random translation
#     translation = np.random.uniform(-0.1, 0.1, 3)
    
#     # Apply transformations
#     vertices = vertices @ rotation_matrix.T
#     vertices *= scale
#     vertices += translation
    
#     return vertices

# def augment_and_save_files(input_folder, output_folder, num_augmentations=5):
#     # Create the output folder if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Loop over all .npz files in the input folder
#     for file_name in os.listdir(input_folder):
#         if file_name.endswith('.npz'):
#             file_path = os.path.join(input_folder, file_name)
#             tensor_data = np.load(file_path)
#             vertices = tensor_data['vertices']
#             faces = tensor_data['faces']
            
#             for i in range(num_augmentations):
#                 # Apply augmentation
#                 augmented_vertices = augment_vertices(vertices)
                
#                 # Combine vertices and faces into a single tensor
#                 augmented_tensor = {
#                     'vertices': augmented_vertices,
#                     'faces': faces
#                 }
                
#                 # Save the augmented tensor to a .npz file
#                 augmented_file_name = file_name.replace('.npz', f'_augmented_{i}.npz')
#                 augmented_file_path = os.path.join(output_folder, augmented_file_name)
#                 np.savez(augmented_file_path, **augmented_tensor)
                
#                 print(f'Successfully augmented and saved: {augmented_file_name}')

# input_folder = '/home/aalab/Desktop/tripoFT/tensors'
# output_folder = '/home/aalab/Desktop/tripoFT/augmented_data'
# augment_and_save_files(input_folder, output_folder)


Split the data

In [21]:
tensor_directory = '/home/aalab/Desktop/tripoFT/tensors'

# Directory to save the splits
train_dir = '/home/aalab/Desktop/tripoFT/split_data/train'
val_dir = '/home/aalab/Desktop/tripoFT/split_data/val'
test_dir = '/home/aalab/Desktop/tripoFT/split_data/test'

# # Create directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Set the seed for reproducibility
random.seed(42)

# List of voxel tensor files
tensor_files = os.listdir(tensor_directory)
random.shuffle(tensor_files)

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate the number of files for each split
num_files = len(tensor_files)
num_train = int(train_ratio * num_files)
num_val = int(val_ratio * num_files)

# Split the files
train_files = tensor_files[:num_train]
val_files = tensor_files[num_train:num_train + num_val]
test_files = tensor_files[num_train + num_val:]

# Move files to respective directories
for file in train_files:
    shutil.copyfile(os.path.join(tensor_directory, file), os.path.join(train_dir, file))

for file in val_files:
    shutil.copyfile(os.path.join(tensor_directory, file), os.path.join(val_dir, file))

for file in test_files:
    shutil.copyfile(os.path.join(tensor_directory, file), os.path.join(test_dir, file))

print("Data split into train, validation, and test sets.")
print(num_files, num_train, num_val)

Data split into train, validation, and test sets.
145 101 21


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [33]:
import numpy as np
import torch

# Load the .npz file
data = np.load('/home/aalab/Desktop/tripoFT/tensors/a_little_bite_of_france..npz')

# Iterate over the loaded arrays
for key in data:
    # Convert the NumPy array to a PyTorch tensor
    tensor = torch.tensor(data[key])
    # Check the device of the tensor
    print(f"Device of '{key}': {tensor.device}")


Device of 'vertices': cpu
Device of 'faces': cpu


Load the Data

Get the TripoSR Model directory

Training Loop